In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data Exploration
our dataset consist of 4 files, 2 for each plant
- power generation data
- weather sensor data


### The power generations data consists of 7 fields:
| DATE_TIME | PLANT_ID | SOURCE_KEY | DC_POWER | AC_POWER | DAILY_YIELD | TOTAL_YIELD |
| --------- | -------- | ---------- | -------- | -------- | ----------- | ----------- |
| Date and time for each observation (%d-%m-%Y, %H:%M). Observations recorded at 15 minute intervals. | Plant ID - this will be common for the entire file. | Source key in this file stands for the inverter id. | Amount of DC power generated by the inverter (source_key) in this 15 minute interval. Units - kW. | Amount of AC power generated by the inverter (source_key) in this 15 minute interval. Units - kW. | Daily yield is a cumulative sum of power generated on that day, till that point in time. | This is the total yield for the inverter till that point in time. |

### The weather data consists of 6 fields:
| DATE_TIME | PLANT_ID | SOURCE_KEY | AMBIENT_TEMPERATURE | MODULE_TEMPERATURE | IRRADIATION |
| --------- | -------- | ---------- | -------- | -------- | ----------- |
| Date and time for each observation (%d-%m-%Y, %H:%M). Observations recorded at 15 minute intervals. | Plant ID - this will be common for the entire file. | Stands for the sensor panel id. This will be common for the entire file because there's only one sensor panel for the plant. | This is the ambient temperature at the plant. | There's a module (solar panel) attached to the sensor panel. This is the temperature reading for that module. | Amount of irradiation for the 15 minute interval. Solar irradiation is the quantity that measures the energy per unit area of ​​incident solar radiation on a surface - the power received during a time (J/m2 or Wh/m2 )  |

Data is gathered every 15 minutes for 34 days.

Expecting 3264 samples per sampling point:

`24 hours x 4 times per hour x 34 days = 3264 samples`

In [4]:
plant_1_gen = pd.read_csv("Plant_1_Generation_data.csv")
plant_2_gen = pd.read_csv("Plant_2_Generation_data.csv")
plant_1_wea = pd.read_csv("Plant_1_Weather_Sensor_data.csv")
plant_2_wea = pd.read_csv("Plant_2_Weather_Sensor_data.csv")

### some samples missing
Grouping by sensore and checking the mean we see that it is lower than expected.

In [5]:

print("plant_1_gen mean samples: ", plant_1_gen.groupby("SOURCE_KEY")["DATE_TIME"].count().mean().round())
print("plant_2_gen mean samples: ", plant_2_gen.groupby("SOURCE_KEY")["DATE_TIME"].count().mean().round())
print("plant_1_wea mean samples: ", plant_1_wea.groupby("SOURCE_KEY")["DATE_TIME"].count().mean().round())
print("plant_2_wea mean samples: ", plant_2_wea.groupby("SOURCE_KEY")["DATE_TIME"].count().mean().round())

plant_1_gen mean samples:  3126.0
plant_2_gen mean samples:  3077.0
plant_1_wea mean samples:  3182.0
plant_2_wea mean samples:  3259.0


In [7]:
plant_1_gen["DC_POWER" == 0].plot()


KeyError: False